# Задание 1
Напишите функцию, которая возвращает название валюты (поле 'Name') с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [36]:
import requests

def max_rate():
    r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
    response = r.json()['Valute'].items()
    a = 0
    v = ''
    for valute, valute_data in response:
        if int(valute_data['Value']) > a:
            a = valute_data['Value']
            v = valute
    return print(v)

In [37]:
max_rate()

CNY


# Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [38]:
import sys
sys.path.append('./libs')
from exchange import Rate

In [41]:
class RateWithDiff(Rate):
    def __init__(self, diff):
        super().__init__()
        self.diff = diff
        self.values = {'EUR': 78}
        
# на примере ЕВРО и пршлого значения, равного 78
           
    def eur(self):
        last_value = self.values.get('EUR')
        self.values['EUR'] = super().eur()
        if self.diff:
            return self.values['EUR'] - last_value
        else:
            return self.values['EUR']
               
a = RateWithDiff(diff=True)
print(a.__dict__)
print(a.eur())
print(a.__dict__)

{'format': 'value', 'diff': True, 'values': {'EUR': 78}}
0.681200000000004
{'format': 'value', 'diff': True, 'values': {'EUR': 78.6812}}


In [46]:
class Rate:
    """
    
    """
    def __init__(self, format='value', diff):
        self.format = format
        self.diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')